In [45]:
from __future__ import annotations
import json
from dataclasses import asdict, dataclass
from typing import Literal
import openai
import sys

openai.api_key = "XX"
MODEL = "gpt-3.5-turbo"

@dataclass
class Message:
    role: Literal["system", "user", "assistant"]
    content: str

    @staticmethod
    def from_response(response: dict) -> Message:
        response["choices"][0]["message"]
        return Message(**(response["choices"][0]["message"]))

    def __str__(self) -> str:
        return f"{self.role}: {self.content}"


class Transcript(list[Message]):
    def raw(self) -> list[dict]:
        return [asdict(m) for m in self]

    def __str__(self) -> str:
        return "\n".join([str(m) for m in self])


def chat_step(transcript_state: Transcript) -> Message:
    # print(transcript_state[-1])
    chat_completion = openai.ChatCompletion.create(
        model=MODEL, messages=transcript_state.raw()
    )
    if not isinstance(chat_completion, dict):
        raise Exception(f"Unexpected response: {chat_completion}")
    response_message = Message.from_response(chat_completion)
    transcript_state.append(response_message)
    # print(response_message)
    #print("transcript_state = \n", transcript_state)
    return response_message


# if __name__ == "__main__":
#     if len(sys.argv) < 2:
#         print("Usage: chat_completion.py <message>")
#         exit(1)
#     transcript = Transcript(
#         [
#             Message("user", " ".join(sys.argv[1:])),
#         ]
#     )
#     chat_step(transcript)

In [5]:
transcript = Transcript(
    [
        Message("user", "Hello")
    ]
)
chat_step(transcript)

user: Hello
assistant: Hi there! How can I assist you today?


Message(role='assistant', content='Hi there! How can I assist you today?')

In [47]:
prompt_template = PromptTemplate.from_template(
"""
Please extract information from the deal proposal market by three backsticks:

#Deal proposal:
```{deal_proposal}```

#Json output format:
{json_schema}

# Reuirementts
1. Output extracted information in the above json format
2. If no information found for that specific column, ignore it.
3. Only output the json object
"""
)

Offline Process
- Convert email/voice transactript into predefined stuctured data for database ingestion
- Summarize the key information from the context
- Find possible match

Online process:
- ad-hoc Semantic search
- 

## Product recommendation based on inital context

In [50]:
## This is generated my OPENAI
deal_proposal = '''
Subject: Inquiry Regarding Wine Product Shipment Logistics

Dear Sarah,

I hope this email finds you well. My name is Tom Hanks, and I represent Walmart, a trusted retailer known for its commitment to quality and service. We are currently in search of a reliable logistics partner to facilitate the transportation of our wine products from our storage facilities to our stores every Friday night.

Our main requirement is that the chosen carrier possesses the necessary permits and qualifications to handle the transportation of wine products. Given the sensitivity and legal regulations associated with alcoholic beverages, it is imperative that we work with a reputable logistics provider who can ensure compliance with all relevant laws and regulations.

We understand that your expertise lies in connecting businesses with suitable carriers, and we believe that your assistance could be invaluable in helping us find the right partner for this specific requirement. Here are some key details about our logistics needs:

Frequency: We require weekly transportation services, specifically on Friday nights, to ensure our stores are well-stocked for the weekend.

Origin and Destination: The shipments will originate from our storage facility, which is located at located in Fremont, CA, and will be delivered to various Walmart stores in the bay area. The specific store destinations will be provided upon further discussion.

Volume: The volume of wine product shipments may vary, so flexibility in accommodating different quantities is essential.

Permits: As mentioned earlier, it is mandatory that the carrier holds all the necessary permits and licenses for the transportation of wine products, and they must be compliant with all applicable regulations.

Timeliness and Reliability: Punctuality and reliability are critical factors for us. We expect the carrier to adhere to agreed-upon schedules and provide efficient transportation services.

In addition to regulatory compliance, we are also mindful of cost-effectiveness. Ideally, we are looking for a carrier that can provide this service within a budget of approximately $1,000 per trip. Cost efficiency is a key factor in our decision-making process.

We would greatly appreciate it if you could provide us with information on carriers in your network who meet these criteria. Additionally, any relevant details regarding their experience in handling wine product shipments would be beneficial.

If you have any questions or require further information from our end, please do not hesitate to reach out to us at [Your Contact Information].

We look forward to your prompt response and hope to establish a successful partnership that ensures the smooth and compliant transportation of our wine products.

Thank you for your time and assistance.

Sincerely,

Tom Hanks
Walmart shipment center
Walmart
tom.hanks@walmart.com
800-675-2025
'''

In [51]:
json_schema = """
{
  "type": "object",
  "properties": {
    "Agent_name": {
      "type": "string",
      "description": "The name of the agent who initialized this proposal."
    },
    "agent_company": {
      "type": "string",
      "description": "The company name of the agent."
    },
    "agent_email": {
      "type": "string",
      "format": "email",
      "description": "The email address of the agent."
    },
    "agent_phone_number": {
      "type": "string",
      "description": "The phone number of the agent."
    },
    "items": {
      "type": "string",
      "description": "The item(s) to be shipped."
    },
    "requirement": {
      "type": "string",
      "description": "Additional requirements or notes."
    },
    "origin_city": {
      "type": "string",
      "description": "The origin of the shipment."
    },
    "destination_city": {
      "type": "string",
      "description": "The destination of the shipment."
    },
    "frequency": {
      "type": "string",
      "description": "The frequency of the shipment."
    },
    "Budget": {
      "type": "number",
      "minimum": 0,
      "description": "The budget associated with the shipment."
    },
    "Other": {
      "type": "string",
      "description": "Any other relevant information."
    }
  },
  "required": [
    "Agent_name",
    "agent_email",
    "items",
    "origin",
    "destination",

  ]
}
"""

### 1. Inquery Generation

Automatically convert inquery into database

In [52]:
from langchain import PromptTemplate

In [53]:
def get_inquery(input_conext, query_schema):
    prompt = prompt_template.format(deal_proposal=input_conext, json_schema=query_schema)
    transcript = Transcript(
        [
            Message("user", prompt)
        ]
    )
    res = chat_step(transcript)
    return res.content

In [55]:
# May need good data valiadtion before pushing into a database
inquery_data = get_inquery(deal_proposal, json_schema)

In [56]:
print(inquery_data)

{
  "Agent_name": "Tom Hanks",
  "agent_company": "Walmart shipment center",
  "agent_email": "tom.hanks@walmart.com",
  "agent_phone_number": "800-675-2025",
  "items": "Wine Product",
  "requirement": "Carrier must possess necessary permits and qualifications to handle wine product transportation. Punctuality and reliability are important.",
  "origin_city": "Fremont, CA",
  "destination_city": "Bay Area",
  "frequency": "Weekly transportation services on Friday nights",
  "Budget": 1000
}


### 2. Inquery Insights

In [116]:
key_requirement_prompt = PromptTemplate.from_template("""
Please summarize the top 5 key requirement form the proposal
                                                      
## Proposal
{proposal}
                                                      
Output the requirement in the format of list of strings

"""
)

In [117]:
prompt = key_requirement_prompt.format(proposal=deal_proposal)
transcript = Transcript(
    [
        Message("user", prompt)
    ]
)
res = chat_step(transcript)

In [118]:
print(res.content)

1. Necessary permits and qualifications for handling wine product transportation.
2. Weekly transportation services on Friday nights.
3. Shipment from Fremont, CA, to various Walmart stores in the bay area.
4. Flexibility in accommodating varying volumes of wine product shipments.
5. Timeliness, reliability, and adherence to agreed-upon schedules.


In [ ]:
carriers = [
    {
        "name": "shipper comany 1",
        "services": "Cross-Border Transportation, Freight Forwarding, Transportation Management"
    },
    {
        "name": "shipper comany 2",
        "services": "Order fulfillment, Warehousing, Inventory management"
    }

]

In [57]:
# get recommendations
# Assume we have a product/caryyier databse 
# query the qualified database with customer's inquery
